In [1]:
# 提供所需要的服务：指标权重服务，关键字排序服务和关键字出价预测。

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import time
import datetime
from sklearn.externals import joblib
from enum import Enum

%matplotlib inline

In [2]:
class KeywordService:
    _indexes = None #Pandas 指标的保存，包括描述、SQL语句、名字和历史权重
    _LastIndexes = None # 上次学习模型考量的指标组合，应copy防止_indexed中删除。
    _SortedKeywords = None #历史关键字排序结果
    _LastTime = None #上次历史计算时间
    _Models4P2C = {} # 各个关键字出价预测历史参数:从出价以及其他特征到点击率
    _Models4R2C = {} # 各个关键字出价预测历史参数:从平均排名以及其他特征到点击率
    _Models4R2P = {} # 各个关键字出价预测历史参数:从平均排名以及其他特征到平均点击价格
    _LastShow = None #最近一个星期的平均展示
    _LastClick = None #最近一个星期的平均点击
    _LastCPC = None #最近一个星期的平均点击价格
    _LastRank = None #最近一个星期的平均排名
    _imp_conv_fittedModel = None # 从指标到潜在订单数的LR模型
    _data = None # 关键字各项总结性信息
    _mean_profit = None # 单个订单平均保费收益（across all keywords）
    
    def buildKeywords(self, keywords, infolist, infotype):
        """
            关键词总结性信息汇总：关键词格式 计划|单元|关键词
        """
        # 从关键字统计信息中，获得关键字的
        for keyword in keywords:
            self._data.loc[keyword,infotype] = infolist[keyword]
    
    def __init__(self):
        """
            服务实例初始化。负责：
            各项数据、模型的建立。
        """
        self._imp_conv_fittedModel = model
        self._data = pd.DataFrame(columns=['keyword' ,'conv','imp_conv']).set_index('keyword')
        self._keywords = []
    
    @staticmethod
    def load(filename):
        """
            从文件中获取该类
        """
        service = joblib.load(filename)
        if isinstance(service, KeywordService):
            return service
        else:
            raise Exception("文件并非KeywordService类持久化文件。")   
    
    @staticmethod
    def save(filename, service):
        """
            dump到文件
        """
        assert isinstance(service, KeywordService)
        joblib.dump(service, filename) 
        
    def collect(self):
        """
            收集：
            1.最近的展示
            2.最近的点击
            3.最近的平均点击价格
            4.单个订单平均保费收益
        """
        keyword_datas = [pd.read_excel("关键字统计数据：出价均价和展示", sheetname="百度pc"),
                         pd.read_excel("关键字统计数据：出价均价和展示", sheetname="百度pc1"),
                         pd.read_excel("关键字统计数据：出价均价和展示", sheetname="百度yd"),
                         pd.read_excel("关键字统计数据：出价均价和展示", sheetname="百度yd1")]
        combined_keyword_data = pd.concat(keyword_datas)
        
        
    
    def learn(self):
        """
            完成所有模型的学习，更新LastTime.
        """
        pass
        
    #——————指标学习部分——————#
    def add_index(self):
        """
            增加新的指标：名称，SQL，描述。
        """
        pass
        
    def delete_index(self):
        """
            删除新的指标
        """
        pass
        
        
    def revise_index(self):
        """
            修改指标信息
        """
        pass
        
    def _learnIndexModel(self, indexes):
        """
            学习指标->订单的模型_imp_conv_fittedModel：
            同时将使用的indexes保存一份副本。
            
        """
        pass
    
    def getIndexTable(self):
        """
            根据模型，获得指标权重排名表。
        """
        pass
        
    def predictImplGain(self):
        """
            对给出指标数据进行预测分析得到潜在订单值。需要检查给出指标和学习指标是相同的。
        """
        pass
        
    #——————出价预测部分——————#
    def _learnP2C(self):
        """
            From Price to Click 模型学习。
        """
        pass
        
    def _learnR2C(self):
        """
            From Rank to Click 模型学习。
        """
        pass
        
    def _learnR2P(self):
        """
            From Rank to Price 模型学习。
        """
        pass
    
    def predictP2ROI(self):
        """
            预估：
            建议_LastTime较久先重新collect再进行预估。
            建议_LastTime很久先重新learn再进行预估。
        """
        pass
        
    def predictP2Gain(self):
        """
            预估：
            建议_LastTime较久先重新collect再进行预估。
            建议_LastTime很久先重新learn再进行预估。
        """
        pass
    
    def predictP2C(self):
        pass
        
    def predictR2C(self):
        pass
    
    def predictR2P(self):
        pass
    
    def predictR2ROI(self):
        """
            预估：
            建议_LastTime较久先重新collect再进行预估。
            建议_LastTime很久先重新learn再进行预估。
        """
        pass
        
    def predictR2Gain(self):
        """
            预估：
            建议_LastTime较久先重新collect再进行预估。
            建议_LastTime很久先重新learn再进行预估。
        """
        pass

In [62]:
keyword_filename = "keywordReport.xlsx"
for sheetname in ["百度pc"]:#, "百度pc1", "百度yd", "百度yd1"]:
    keywords = []
    keyword_data = pd.read_excel(keyword_filename, sheetname=sheetname)
    keyword_data = keyword_data[(~(keyword_data["关键词"].str.contains("已删除",na=True))) &
                                (~(keyword_data["推广计划"].str.contains("已删除",na=True))) &
                                (~(keyword_data["推广单元"].str.contains("已删除",na=True))) &
                                ((pd.Timestamp.today()-keyword_data["日期"])<pd.to_timedelta("777 days"))]
    keyword_data["排名总量"] = keyword_data["平均排名"] * keyword_data["展现"]
    keyword_data = keyword_data.groupby(["关键词","推广单元","推广计划"]).agg(
        {"展现": "sum","点击": "sum","排名总量":"sum","消费":"sum"}).reset_index()
    keyword_data["平均点击价格"] = np.where(keyword_data["点击"]==0,0,keyword_data["消费"]/keyword_data["点击"])
    keyword_data["平均排名"] = keyword_data["排名总量"]/keyword_data["展现"]
    keyword_data["点击率"] = keyword_data["点击"]/keyword_data["展现"]
    new_keywords = keyword_data["推广计划"].str.cat(keyword_data["推广单元"],sep="|").str.cat(keyword_data["关键词"],sep="|").values
    assert len(set(new_keywords) & set(keywords)) == 0
    keywords.extend(new_keywords)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [5]:

keyword_data = pd.read_excel("keywordReport.xlsx")

In [20]:
pd.Timestamp.today

<bound method Timestamp.today of <class 'pandas._libs.tslibs.timestamps.Timestamp'>>

In [35]:
a=[]

In [69]:

keyword_data = pd.read_excel("keywordReport.xlsx", sheetname="百度pc")
keyword_data = keyword_data[(~(keyword_data["关键词"].str.contains("已删除",na=True))) &
                            (~(keyword_data["推广计划"].str.contains("已删除",na=True))) &
                            (~(keyword_data["推广单元"].str.contains("已删除",na=True))) &
                            ((pd.Timestamp.today()-keyword_data["日期"])<pd.to_timedelta("777 days"))]
keyword_data["排名总量"] = keyword_data["平均排名"] * keyword_data["展现"]
keyword_data = keyword_data.groupby(["关键词","推广单元","推广计划"]).agg(
    {"展现": "sum","点击": "sum","排名总量":"sum","消费":"sum"}).reset_index()
keyword_data["平均点击价格"] = np.where(keyword_data["点击"]==0,0,keyword_data["消费"]/keyword_data["点击"])
keyword_data["平均排名"] = keyword_data["排名总量"]/keyword_data["展现"]
keyword_data["点击率"] = keyword_data["点击"]/keyword_data["展现"]
new_keywords = keyword_data["推广计划"].str.cat(keyword_data["推广单元"],sep="|").str.cat(keyword_data["关键词"],sep="|").values
assert len(set(new_keywords) & set(keywords)) == 0
keywords.extend(new_keywords)

KeyboardInterrupt: 

In [68]:
keyword_data

,关键词,推广单元,推广计划,消费,排名总量,展现,点击,平均点击价格,平均排名,点击率
